## Reprojecting Raster Data

#### Imports

In [1]:
import rasterio
from rasterio.warp import calculate_default_transform, reproject

#### Calling the input data

In [2]:
raster_file1 = 'germany_population.tif'
raster_file2 = 'germany_elevation.tif'

In [3]:
with rasterio.open(raster_file1) as src1:
    band1 = src1.read(1)
    meta1 = src1.meta

In [4]:
with rasterio.open(raster_file2) as src2:
    band2 = src2.read(1)
    meta2 = src2.meta

In [5]:
meta1

{'driver': 'GTiff',
 'dtype': 'float64',
 'nodata': None,
 'width': 1101,
 'height': 936,
 'count': 1,
 'crs': CRS.from_wkt('GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AXIS["Latitude",NORTH],AXIS["Longitude",EAST],AUTHORITY["EPSG","4326"]]'),
 'transform': Affine(0.00833333330032682, 0.0, 5.850416665888758,
        0.0, -0.008333333299795073, 55.066249981282525)}

In [6]:
meta2

{'driver': 'GTiff',
 'dtype': 'int16',
 'nodata': None,
 'width': 11004,
 'height': 9354,
 'count': 1,
 'crs': CRS.from_wkt('GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AXIS["Latitude",NORTH],AXIS["Longitude",EAST],AUTHORITY["EPSG","4326"]]'),
 'transform': Affine(0.0008333333333333301, 0.0, 5.85208333333333,
        0.0, -0.0008333333333333301, 55.06541666666667)}

#### Reprojecting to local CRS

In [7]:
target_crs = 'EPSG:25832'

In [8]:
def reproject_raster(input_path, output_path, target_crs):
    # Open the input raster file
    with rasterio.open(input_path) as src:
        # Calculate the transform and dimensions for the target CRS
        transform, width, height = calculate_default_transform(
            src.crs, target_crs, src.width, src.height, *src.bounds)
        
        # Update metadata with new CRS and dimensions
        metadata = src.meta.copy()
        metadata.update({
            'crs': target_crs,
            'transform': transform,
            'width': width,
            'height': height
        })
        
        # Create the output raster file with the new CRS
        with rasterio.open(output_path, 'w', **metadata) as dst:
            # Reproject the data
            for i in range(1, src.count + 1):  # Loop over each band
                reproject(
                    source=rasterio.band(src, i),
                    destination=rasterio.band(dst, i),
                    src_transform=src.transform,
                    src_crs=src.crs,
                    dst_transform=transform,
                    dst_crs=target_crs
                )



In [9]:
# Paths to the input and output files
reprojected_file1 = 'germany_population_reprojected.tif'
reprojected_file2 = 'germany_elevation_reprojected.tif'

# Reproject both raster files
reproject_raster(raster_file1, reprojected_file1, target_crs)
reproject_raster(raster_file2, reprojected_file2, target_crs)

In [10]:
with rasterio.open(reprojected_file1) as src1:
    band1 = src1.read(1)  # Read the data from the first raster
    meta1 = src1.meta  # Metadata for the Europe raster

with rasterio.open(reprojected_file2) as src2:
    band2 = src2.read(1)
    meta2 = src2.meta  # Metadata for the Germany raster

print(meta1['crs'])
print(meta2['crs'])

EPSG:25832
EPSG:25832
